In [1]:
import pandas as pd
from pandarallel import pandarallel
import numpy as np
import seaborn as sns
import plotly.express as px
import torch
import os
from tqdm import tqdm
import pickle
import warnings
from Levenshtein import distance as lev_distance
from sentence_transformers import SentenceTransformer

from pullenti_wrapper.langs import set_langs, RU
from pullenti_wrapper.processor import Processor, GEO, ADDRESS
from pullenti_wrapper.referent import Referent

set_langs([RU])
warnings.filterwarnings('ignore')
sns.set(rc={'figure.figsize': (20, 10), 'figure.facecolor': 'white'})
sns.set_palette("viridis")
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pandarallel.initialize(progress_bar=True)
os.environ["TOKENIZERS_PARALLELISM"] = "true"  # activate parallelism
processor = Processor([GEO, ADDRESS])

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


# Stage 1 - Get and preprocess Data

In [2]:
def join_non_null_values(l):
    res = [str(x) for x in l if str(x) != 'nan']
    return ', '.join(res)

In [3]:
def coalesce(s: pd.Series, *series):
    """coalesce the column information like a SQL coalesce."""
    for other in series:
        s = s.mask(pd.isnull, other)
    return s

In [4]:
# recursive function
def get_ner_elements(referent, level=0):
    tmp = {}
    a = ""
    b = ""
    for key in referent.__shortcuts__:
        value = getattr(referent, key)
        if value in (None, 0, -1):
            continue
        if isinstance(value, Referent):
            get_ner_elements(value, level + 1)
        else:
            if key == 'type':
                a = value
            if key == 'name':
                b = value
                # print('ok', value)
            if key == 'house':
                a = "дом"
                b = value
                tmp[a] = b
            if key == 'flat':
                a = "квартира"
                b = value
                # print('ok', value)
                tmp[a] = b
            if key == 'building':
                a = "Литера"
                b = value
                tmp[a] = b
        if key == 'corpus':
            a = "корпус"
            b = value
            tmp[a] = b
    tmp[a] = b
    addr.append(tmp)
    return addr

In [5]:
def get_address_ner_objects(address: str):
    global addr  # we need to declare global variable to change it
    processor = Processor([GEO, ADDRESS])
    addr = []
    try:
        result = processor(address)
        referent = result.matches[0].referent
        ner_model_res = get_ner_elements(referent)
        merged_data = {k: v for d in ner_model_res for k, v in d.items()}
        data = pd.DataFrame([merged_data])
        return merged_data
    except:
        return []

In [12]:
train = (
    pd.read_csv('../data/raw/additional_data/building_20230808.csv').add_suffix('_building')
    .merge(
        pd.read_csv('../data/raw/additional_data/district_20230808.csv').add_suffix('_district'),
        left_on='district_id_building',
        right_on='id_district',
        how='left',
    )
    .merge(
        pd.read_csv('../data/raw/additional_data/prefix_20230808.csv').add_suffix('_prefix'),
        left_on='prefix_id_building',
        right_on='id_prefix',
        how='left',
    )
    .merge(
        pd.read_csv('../data/raw/additional_data/town_20230808.csv').add_suffix('_town'),
        left_on='town_id_prefix',
        right_on='id_town',
        how='left',
    )
    .merge(
        pd.read_csv('../data/raw/additional_data/geonim_20230808.csv').add_suffix('_geonim'),
        left_on='geonim_id_prefix',
        right_on='id_geonim',
        how='left',
    )
    .merge(
        pd.read_csv('../data/raw/additional_data/geonimtype_20230808.csv').add_suffix('_geonimtype'),
        left_on='type_id_geonim',
        right_on='id_geonimtype',
        how='left',
    )
    .merge(
        pd.read_csv('../data/raw/additional_data/area_20230808.csv').add_suffix('_area'),
        left_on='area_id_prefix',
        right_on='id_area',
        how='left',
    )
    .merge(
        pd.read_csv('../data/raw/additional_data/areatype_20230808.csv').add_suffix('_areatype'),
        left_on='type_id_area',
        right_on='id_areatype',
        how='left',
    )
    .assign(name_district_full=lambda df_: df_.name_district + ' район')
    .assign(
        all_in_field=lambda df_: df_[['full_address_building', 'type_building',
                                      'name_district_full', 'name_area', 'name_areatype']]
        .apply(lambda x: join_non_null_values(x), axis=1)
    )
    .assign(miscellaneous_objects = lambda df_: df_.parallel_apply(lambda x: get_address_ner_objects(x['full_address_building']), axis=1))
    
)
train = pd.concat([train.drop(['miscellaneous_objects'], axis=1), train['miscellaneous_objects'].apply(pd.Series)], axis=1)

train['ner_city'] = coalesce(train['short_name_town'], train['город'], train['поселок'], train['деревня'], train['село'],
                             train['поселок городского типа'])
train['ner_street'] = coalesce(train['only_name_geonim'], train['улица'], train['проспект'], train['набережная'], train['шоссе'], train['парк'],
                               train['переулок'], train['площадь'], train['аллея'], train['линия'], train['автодорога'],
                               train['проезд'], train['бульвар'])
train['ner_house'] = coalesce(train['house_building'], train['дом'])
train['ner_corpus'] = coalesce(train['corpus_building'], train['корпус'])
train['ner_liter'] = coalesce(train['liter_building'], train['Литера'])
train['ner_district'] = coalesce(train['name_district'], train['муниципальный район'], train['микрорайон'], train['квартал'], train['район'],
                                 train['муниципальный округ'])
train['ner_area'] = coalesce(train['name_area'], train['станция'], train['территория'], train['мост'], train['тупик'])
train['ner_subject'] = coalesce(train['область'], train['округ'], train['волость'])

train.to_pickle("../data/processed/main_data.pkl")
train

,id_building,prefix_id_building,district_id_building,house_building,corpus_building,liter_building,villa_building,parcel_building,full_address_building,is_updated_building,is_actual_building,type_building,municipality_id_building,short_address_building,post_prefix_building,build_number_building,id_district,name_district,is_updated_district,is_actual_district,id_prefix,town_id_prefix,geonim_id_prefix,area_id_prefix,toponim_id_prefix,name_prefix,short_name_prefix,search_index_prefix,is_updated_prefix,is_actual_prefix,sub_rf_id_prefix,has_buildings_prefix,id_town,name_town,short_name_town,search_index_town,is_updated_town,is_actual_town,has_buildings_town,id_geonim,type_id_geonim,name_geonim,short_name_geonim,is_updated_geonim,is_actual_geonim,only_name_geonim,id_geonimtype,name_geonimtype,short_name_geonimtype,is_updated_geonimtype,is_actual_geonimtype,id_area,type_id_area,name_area,short_name_area,is_updated_area,is_actual_area,only_name_area,id_areatype,name_areatype,short_name_areatype,is_updated_areatype,is_actual_areatype,name_district_full,all_in_field,город,улица,дом,поселок,проспект,корпус,Литера,муниципальный район,набережная,шоссе,парк,переулок,площадь,аллея,,линия,автодорога,микрорайон,деревня,проезд,квартал,бульвар,станция,район,территория,муниципальный округ,мост,тупик,область,село,поселок городского типа,округ,волость,ner_city,ner_street,ner_house,ner_corpus,ner_liter,ner_district,ner_area,ner_subject
0,56343,11132,35,12,NaN,А,NaN,NaN,"город Пушкин, Кедринская улица, дом 12",True,False,NaN,107.0,"г.Пушкин, Кедринская ул., д. 12",NaN,NaN,35,Пушкинский,True,True,11132,28.0,4801.0,NaN,NaN,"город Пушкин, Кедринская улица","г.Пушкин, Кедринская ул.","'город':1,5 'кедринск':7 'кедринская':3 'пушкин':2,6 'улиц':8 'улица':4",True,True,15.0,True,28.0,город Пушкин,г.Пушкин,"'город':1,3 'пушкин':2,4",True,True,True,4801.0,12.0,Кедринская улица,Кедринская ул.,True,True,Кедринская,12.0,улица,ул.,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Пушкинский район,"город Пушкин, Кедринская улица, дом 12, Пушкинский район",ПУШКИН,КЕДРИНСКАЯ,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,г.Пушкин,Кедринская,12,NaN,А,Пушкинский,NaN,NaN
1,595,6987,38,4Б,NaN,NaN,NaN,NaN,"поселок Ушково, Пляжевая улица, дом 4Б",True,False,NaN,128.0,"пос. Ушково, Пляжевая ул., д. 4Б",NaN,NaN,38,Курортный,True,True,6987,46.0,3.0,NaN,NaN,"посёлок Ушково, Пляжевая улица","пос. Ушково, Пляжевая ул.",'пляжев':7 'пляжевая':3 'поселок':5 'посёлок':1 'улиц':8 'улица':4 'ушков':6 'ушково':2,True,True,15.0,True,46.0,посёлок Ушково,пос. Ушково,'поселок':3 'посёлок':1 'ушков':4 'ушково':2,True,True,True,3.0,12.0,Пляжевая улица,Пляжевая ул.,True,True,Пляжевая,12.0,улица,ул.,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Курортный район,"поселок Ушково, Пляжевая улица, дом 4Б, Курортный район",NaN,ПЛЯЖЕВАЯ,4Б,УШКОВО,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,пос. Ушково,Пляжевая,4Б,NaN,NaN,Курортный,NaN,NaN
2,7134,6469,15,30,2,Е,NaN,NaN,"г.Санкт-Петербург, проспект Маршала Жукова, дом 30, корпус 2, литера Е",True,False,Нежилое,30.0,"г.Санкт-Петербург, пр. Маршала Жукова, д. 30, к. 2, л. Е",198303,NaN,15,Кировский,True,True,6469,36.0,2194.0,NaN,NaN,"г.Санкт-Петербург, проспект Маршала Жукова","г.Санкт-Петербург, пр. Маршала Жукова","'г':1,8 'жуков':14 'жукова':7 'марша':13 'маршала':6 'петербург':4,11 'проспект':5,12 'санкт':3,10 'санкт-петербург':2,9",True,True,15.0,True,36.0,г.Санкт-Петербург,г.Санкт-Петербург,"'г':1,5 'петербург':4,8 'санкт':3,7 'санкт-петербург':2,6",True,True,True,2194.0,10.0,проспект Маршала Жукова,пр. Маршала Жукова,True,True,Маршала Жукова,10.0,проспект,пр.,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Кировский район,"г.Санкт-Петербург, проспект Маршала Жукова, дом 30, корпус 2, литера Е, Нежилое, Кировский район",САНКТ-ПЕТЕРБУРГ,NaN,30,NaN,МАРШАЛА ЖУКОВА,2,Е,NaN,NaN

In [13]:
# id_building are not the same as indices! Therefor we have to map in at some points 
train[['id_building']].to_pickle("../data/processed/mapping_id_building_index.pkl")

In [6]:
train = pd.read_pickle("../data/processed/main_data.pkl")

In [17]:
for_embeddings = (
    pd.DataFrame(
        np.concatenate([
            train[['id_building', 'full_address_building']].values,
            # train[['id_building', 'all_in_field']].values,
            # train[['id_building', 'short_address_building']].values,
            # train[['id_building', 'name_prefix']].values,
            # train[['id_building', 'short_name_prefix']].values,
            # train[['id_building', 'name_town']].values,
            # train[['id_building', 'short_name_town']].values,
            # train[['id_building', 'name_geonim']].values,
            # train[['id_building', 'short_name_geonim']].values,
            # train[['id_building', 'only_name_geonim']].values,
            # train[['id_building', 'name_prefix']].values,
            # train[['id_building', 'short_name_prefix']].values,
            # train[['id_building', 'name_geonimtype']].values,
            # train[['id_building', 'only_name_area']].values,
            # train[['id_building', 'name_areatype']].values,
            # train[['id_building', 'short_name_areatype']].values,
            # train[['id_building', 'name_district']].values,
            # train[['id_building', 'name_district_full']].values,
        ], axis=0),
        columns=['id_building', 'address']
    )
    .dropna()
    .reset_index(drop=True)
)

In [18]:
for_embeddings.to_pickle('../data/processed/for_embeddings_with_names.pkl')
for_embeddings

,id_building,address
0,56343,"город Пушкин, Кедринская улица, дом 12"
1,595,"поселок Ушково, Пляжевая улица, дом 4Б"
2,7134,"г.Санкт-Петербург, проспект Маршала Жукова, дом 30, корпус 2, литера Е"
3,124415,"поселок Белоостров, Дюны, Центральная улица, дом 5, корпус 2"
4,185368,"поселок Песочный, Речная улица, дом 28, литера Б"
...,...,...
166640,18941,"г.Санкт-Петербург, Бородинская улица, дом 3, литера В"
166641,18942,"посёлок Александровская, 2-я линия, дом 34/3, литера А"
166642,18943,"посёлок Ушково, Детская улица, дом 39"
166643,18944,"г.Санкт-Петербург, Заусадебная улица, дом 25, литера Б"


# Step 2 - Get Embeddings

In [ ]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
# for col in tqdm([col for col in train if col.startswith('ner')]):
l = ['ner_city',
'ner_street',
'ner_house',
'ner_corpus',
'ner_liter',
'ner_district',
'ner_area',
'ner_subject']
# for col in tqdm([col for col in train if col.startswith('ner')]):
for col in tqdm(l):
    data = train[col]
    col_embeddings = model.encode(data.astype('str'),
                              convert_to_tensor=True,
                              show_progress_bar=True,
                              batch_size=1)  # when batch_size > 1 the results are being shuffled
    save_to = '../data/processed/embeddings_{}.pkl'.format(col)
    with open(save_to, 'wb') as fp:
        pickle.dump(col_embeddings, fp)

  0%|          | 0/8 [00:00<?, ?it/s]

Batches:   0%|          | 0/166645 [00:00<?, ?it/s]

 12%|█▎        | 1/8 [21:33<2:30:55, 1293.59s/it]

Batches:   0%|          | 0/166645 [00:00<?, ?it/s]

 25%|██▌       | 2/8 [42:04<2:05:39, 1256.63s/it]

Batches:   0%|          | 0/166645 [00:00<?, ?it/s]

 38%|███▊      | 3/8 [57:19<1:31:43, 1100.65s/it]

Batches:   0%|          | 0/166645 [00:00<?, ?it/s]

 50%|█████     | 4/8 [1:12:06<1:07:46, 1016.50s/it]

Batches:   0%|          | 0/166645 [00:00<?, ?it/s]

In [70]:
n = 140
emb_v1 = model.encode(train['ner_street'].iloc[n], convert_to_tensor=True, show_progress_bar=True, batch_size=128)
# emb_v1 = model.encode(np.array(train.query("id_building == 76055").ner_street), convert_to_tensor=True, show_progress_bar=True, batch_size=128)
# emb_v2 = embeddings_ner_street[train.query("id_building == 76055").index]
with open('../data/processed/embeddings_ner_street.pkl', 'rb') as f:
    embeddings_ner_street = pickle.load(f)
emb_v2 = embeddings_ner_street[n]
emb_v1 == emb_v2

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, Tr

In [76]:
embeddings = model.encode(for_embeddings['address'].astype('str'), convert_to_tensor=True, show_progress_bar=True,
                          batch_size=1)

Batches:   0%|          | 0/166645 [00:00<?, ?it/s]

In [21]:
with open('../data/processed/embeddings.pkl', 'wb') as fp:
    pickle.dump(embeddings, fp)

# Step 3 - Similarity Matcher

In [25]:
with open('../data/processed/embeddings.pkl', 'rb') as f:
    embeddings = pickle.load(f)

for_embeddings = pd.read_pickle("../data/processed/for_embeddings_with_names.pkl")

In [26]:
QUERY = ["г.Санкт-Петербург, набережная Обводного канала, дом 205, литера М"]

In [27]:
def get_best_matches(query, top_n, embeddings: np.array, sentences):
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    query_embedding = model.encode(query, convert_to_tensor=True)
    scores = embeddings.dot(query_embedding.T).ravel()
    best = np.argpartition(scores, -top_n)[-top_n:]
    # mapping = pd.read_pickle("../data/processed/mapping_id_building_index.pkl")
    # raw_ranking_indices = mapping.query("id_building in @raw_ranking_ids").index
    sentences['scores'] = scores
    sentences.loc[list(best), 'lev_distance'] = sentences.loc[list(best), 'address'].apply(
        lambda x: lev_distance(query, x))
    best_matches = sentences[['id_building', 'address', 'scores', 'lev_distance']].loc[list(best)]
    best_matches_within_id_indices = best_matches.groupby(['id_building']).scores.transform(max) == best_matches.scores
    best_matches = best_matches.loc[best_matches_within_id_indices]#.sort_values(by='scores',
                                                                                # ascending=False).reset_index(drop=True)
    if len(best_matches[best_matches['scores'] >= 0.99]):
        return best_matches[best_matches['scores'] >= 0.99]
    elif len(best_matches[best_matches['scores'] >= 0.90]):
        return best_matches[best_matches['scores'] >= 0.90]
    elif len(best_matches[best_matches['scores'] >= 0.90]):
        return best_matches[best_matches['scores'] >= 0.90]
    else:
        return best_matches

In [28]:
QUERY = "196641, Санкт-Петербург г, Школьная ул, д.5"
get_best_matches(QUERY, 10, np.array(embeddings), for_embeddings)

,id_building,address,scores,lev_distance
101331,99808,"г.Санкт-Петербург, Лиговский проспект, дом 60-62",0.813415,33.0
14582,9262,"г.Санкт-Петербург, Лиговский проспект, дом 60-62",0.813415,33.0
107944,106759,"г.Санкт-Петербург, Лиговский проспект, дом 60-62",0.813415,33.0
70971,68247,"г.Санкт-Петербург, Лиговский проспект, дом 60-62",0.813415,33.0
19112,13994,"г.Санкт-Петербург, Лиговский проспект, дом 60-62",0.813415,33.0
78451,76055,"г.Санкт-Петербург, Школьная улица, дом 77",0.831581,18.0
166144,40334,"г.Санкт-Петербург, Лиговский проспект, дом 60-62",0.813415,33.0
91078,89171,"г.Санкт-Петербург, Школьная улица, дом 76",0.816349,18.0
61150,57875,"г.Санкт-Петербург, Шуваловский проспект, дом 61",0.820311,32.0
9578,4040,"г.Санкт-Петербург, Лиговский проспект, дом 60-62",0.813415,33.0


In [30]:
query1="196641, Санкт-Петербург г, Школьная ул, д.5"
query2="г.Санкт-Петербург, Лиговский проспект, дом 60-62"
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
query_embedding1 = model.encode(query1, convert_to_tensor=True)
query_embedding2 = model.encode(query2, convert_to_tensor=True)
query_embedding1.dot(query_embedding2.T).ravel()
# transformer required fine-tuning

tensor([0.8134])

In [31]:
QUERY = ["г.Санкт-Петербург, набережная Обводного канала, дом 205, литера М",
         'г.Санкт-Петербург, набережная Обводного канала, дом 205, литера А']

def multiple_best_matches(file, top_n, embeddings: np.array, sentences):
    res = pd.DataFrame()
    for el in file:
        res = pd.concat([res, get_best_matches(el, top_n, embeddings, sentences)])
    return res.reset_index(drop=True)
    

In [32]:
multiple_best_matches(train['full_address_building'].loc[:10].values, 10, np.array(embeddings), for_embeddings)

,id_building,address,scores,lev_distance
0,56343,"город Пушкин, Кедринская улица, дом 12",1.000000,0.0
1,595,"поселок Ушково, Пляжевая улица, дом 4Б",1.000000,0.0
2,4649,"г.Санкт-Петербург, проспект Маршала Жукова, дом 30, корпус 2, литера А",0.990053,1.0
3,199215,"г.Санкт-Петербург, проспект Маршала Жукова, дом 30, корпус 2, литера Е",1.000000,0.0
4,61406,"г.Санкт-Петербург, проспект Маршала Жукова, дом 30, корпус 2, литера В",0.990869,1.0
5,39404,"г.Санкт-Петербург, проспект Маршала Жукова, дом 30, корпус 2, литера И",0.991955,1.0
6,7134,"г.Санкт-Петербург, проспект Маршала Жукова, дом 30, корпус 2, литера Е",1.000000,0.0
7,39211,"г.Санкт-Петербург, проспект Маршала Жукова, дом 30, корпус 2, литера Д",0.991091,1.0
8,124415,"поселок Белоостров, Дюны, Центральная улица, дом 5, корпус 2",1.000000,0.0
9,100211,"посёлок Песочный, Речная улица, дом 28, литера Б",1.000000,1.0


In [119]:
# def slow_get_best_matches(query, top_n, embeddings: np.array, sentences):
#     model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
#     query_embedding = model.encode(query, convert_to_tensor=True)
#     scores = embeddings.dot(query_embedding.T).ravel()
#     best = np.argpartition(scores, -top_n)[-top_n:]
#     sentences['scores'] = scores
#     sentences.loc[list(best), 'lev_distance'] = sentences.loc[list(best), 'address'].apply(
#         lambda x: lev_distance(query, x))
#     best_matches = sentences[['id_building', 'address', 'scores', 'lev_distance']].loc[list(best)]
#     add_data = pd.read_pickle("../data/processed/main_data.pkl")
#     best_matches = best_matches.merge(add_data[['id_building', 'liter_building', 'name_district_full', 'name_town']])
#     best_matches_within_id_indices = best_matches.groupby(['id_building']).scores.transform(max) == best_matches.scores
#     best_matches = best_matches.loc[best_matches_within_id_indices]#.sort_values(by='scores',
#                                                                                 # ascending=False).reset_index(drop=True)
# 
#     # add logic to topn selection
#     # add catboost here to ass probabilities
#     if len(best_matches[best_matches['scores'] >= 0.98]):
#         return best_matches[best_matches['scores'] >= 0.98]
#     elif len(best_matches[best_matches['scores'] >= 0.90]):
#         return best_matches[best_matches['scores'] >= 0.90]
#     else:
#         return best_matches

# slow_get_best_matches(QUERY, 10, np.array(embeddings), for_embeddings)

# Step 4 NER + 1 one ML ALgo

In [33]:
addr = []
result = processor('город Пушкин, Екатерининский парк, дом б/н, литера Ц')
referent = result.matches[0].referent
x = get_ner_elements(referent)
print(addr)
print(x)

[{'город': 'ПУШКИН'}, {'парк': 'ЕКАТЕРИНИНСКИЙ'}, {'дом': '0', 'Литера': 'Ц'}]
[{'город': 'ПУШКИН'}, {'парк': 'ЕКАТЕРИНИНСКИЙ'}, {'дом': '0', 'Литера': 'Ц'}]


In [34]:
get_address_ner_objects('195256, Санкт-Петербург г., Верности ул., д.28, Строение 2') #город Москва, Екатерининский парк, дом б/н, литера Ц

{'город': 'САНКТ-ПЕТЕРБУРГ', 'улица': 'ВЕРНОСТИ', 'дом': '28', 'Литера': '2'}

test:
г.Санкт-Петербург, дом 223-225, наб. Обводного канала
г.Санкт-Петербург, дом 223-225
посёлок Солнечное, Комсомольская улица
г.Санкт-Петербург, проспект Динамо, дом 12

# Step 5. Finalization with NER and Catboost

In [35]:
def input_transformation(user_input):
    user_input = pd.DataFrame(columns=['город', 'улица', 'дом', 'поселок', 'проспект',
                          'корпус', 'Литера', 'муниципальный район', 'набережная', 'шоссе',
                          'парк', 'переулок', 'площадь', 'аллея', '', 'линия', 'автодорога',
                          'микрорайон', 'деревня', 'проезд', 'квартал', 'бульвар', 'станция',
                          'район', 'территория', 'муниципальный округ', 'мост', 'тупик',
                          'область', 'село', 'поселок городского типа', 'округ', 'волость'])  # rewrite
    user_res = pd.concat([user_input, pd.DataFrame(get_address_ner_objects(QUERY), index=[0])])
    user_res['ner_city'] = coalesce(user_res['город'], user_res['поселок'], user_res['деревня'], user_res['село'],
                                    user_res['поселок городского типа'])
    user_res['ner_street'] = coalesce(user_res['улица'], user_res['проспект'], user_res['набережная'], user_res['шоссе'], user_res['парк'],
                                      user_res['переулок'], user_res['площадь'], user_res['аллея'], user_res['линия'], user_res['автодорога'],
                                      user_res['проезд'], user_res['бульвар'])
    user_res['ner_house'] = user_res['дом']
    user_res['ner_corpus'] = user_res['корпус']
    user_res['ner_liter'] = user_res['Литера']
    user_res['ner_district'] = coalesce(user_res['муниципальный район'], user_res['микрорайон'], user_res['квартал'], user_res['район'],
                                        user_res['муниципальный округ'])
    user_res['ner_area'] = coalesce(user_res['станция'], user_res['территория'], user_res['мост'], user_res['тупик'])
    user_res['ner_subject'] = coalesce(user_res['область'], user_res['округ'], user_res['волость'])
    user_res = user_res[[col for col in user_res if col.startswith('ner')]]
    return user_res

In [37]:
QUERY = "г.Санкт-Петербург, набережная Обводного канала, дом 205, литера М"
input_transformation(QUERY)

,ner_city,ner_street,ner_house,ner_corpus,ner_liter,ner_district,ner_area,ner_subject
0,САНКТ-ПЕТЕРБУРГ,ОБВОДНОГО КАНАЛА,205,NaN,М,NaN,NaN,NaN


In [38]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
query_embedding = model.encode('ОБВОДНОГО КАНАЛА', convert_to_tensor=True, normalize_embeddings=True)

In [39]:
get_best_matches(QUERY, 10, np.array(embeddings), for_embeddings)

,id_building,address,scores,lev_distance
86977,84908,"г.Санкт-Петербург, набережная Обводного канала, дом 205, литера М",1.000000,0.0
35884,31542,"г.Санкт-Петербург, набережная Обводного канала, дом 205, литера А",0.990537,1.0


In [40]:
with open('../data/processed/embeddings.pkl', 'rb') as f:
    embeddings = pickle.load(f)

In [41]:
with open('../data/processed/embeddings_ner_street.pkl', 'rb') as f:
    embeddings_ner_street = pickle.load(f)

In [42]:
# embeddings_ner_street

In [72]:
# emb_v1 = model.encode(np.array(train.query("id_building == 76055").ner_street), convert_to_tensor=True, show_progress_bar=True, batch_size=128)
# emb_v2 = embeddings_ner_street[train.query("id_building == 76055").index]
# emb_v1 == emb_v2

In [232]:
# with open('../data/processed/embeddings_ner_street.pkl'.format(col), 'rb') as f:
#     db_col_embedding = np.array(pickle.load(f)[78451])
# 
# query_col_embedding = model.encode('ШКОЛЬНАЯ', convert_to_tensor=True, show_progress_bar=True,
#                                    batch_size=128)
# db_col_embedding.dot(query_col_embedding.T).ravel()

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

array([1.], dtype=float32)

In [75]:
bin = []
def get_advanced_matching_scores(input_query, embeddings: np.array, sentences):
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    raw_ranking = get_best_matches(input_query, 3, embeddings, sentences)
    # display(raw_ranking)
    raw_ranking_ids = raw_ranking.id_building
    query = input_transformation(input_query).dropna(axis=1)
    mapping = pd.read_pickle("../data/processed/mapping_id_building_index.pkl")
    raw_ranking_indices = mapping.query("id_building in @raw_ranking_ids").index
    global bin
    bin.append(raw_ranking_indices)
    # input_transformation(QUERY)
    # for indx, row in train.iloc[raw_ranking_indices].iterrows():
    #     query_col_embedding = model.encode(col_value, convert_to_tensor=True, normalize_embeddings=True)
    #     db_col_embedding = input_transformation(row.full_address_building).dropna(axis=1)
    #     raw_ranking.loc[:, 'score_'+col] = db_col_embedding.dot(query_col_embedding.T).ravel()
    for col in query.columns:
        col_value = query[col].values[0]
        query_col_embedding = model.encode(col_value, convert_to_tensor=True, normalize_embeddings=True)
        # db_col_embedding = train.loc[list(raw_ranking_indices), col]
        # db_col_embedding = np.array(model.encode(train.loc[list(raw_ranking_indices), col].astype('str').str.upper().values,
        #                                 convert_to_tensor=True,
        #                                 # show_progress_bar=True,
        #                                 batch_size=128))
        with open('../data/processed/embeddings_{}.pkl'.format(col), 'rb') as f:
            db_col_embedding = np.array(pickle.load(f)[list(raw_ranking_indices)])
        raw_ranking.loc[:, 'score_'+col] = db_col_embedding.dot(query_col_embedding.T).ravel()
    raw_ranking.loc[:, 'original_query'] = input_query
    return raw_ranking

QUERY = "196641, Санкт-Петербург г, Школьная ул, д.5"
get_advanced_matching_scores(QUERY, np.array(embeddings), for_embeddings)

,id_building,address,scores,lev_distance,score_ner_city,score_ner_street,score_ner_house,original_query
91078,89171,"г.Санкт-Петербург, Школьная улица, дом 76",0.816349,18.0,0.918204,0.764108,0.478460,"196641, Санкт-Петербург г, Школьная ул, д.5"
61150,57875,"г.Санкт-Петербург, Шуваловский проспект, дом 61",0.820311,32.0,0.918204,1.000000,0.483058,"196641, Санкт-Петербург г, Школьная ул, д.5"
78451,76055,"г.Санкт-Петербург, Школьная улица, дом 77",0.831581,18.0,0.918204,1.000000,0.532029,"196641, Санкт-Петербург г, Школьная ул, д.5"


In [262]:
train.loc[list(bin[0])]

,id_building,prefix_id_building,district_id_building,house_building,corpus_building,liter_building,villa_building,parcel_building,full_address_building,is_updated_building,is_actual_building,type_building,municipality_id_building,short_address_building,post_prefix_building,build_number_building,id_district,name_district,is_updated_district,is_actual_district,id_prefix,town_id_prefix,geonim_id_prefix,area_id_prefix,toponim_id_prefix,name_prefix,short_name_prefix,search_index_prefix,is_updated_prefix,is_actual_prefix,sub_rf_id_prefix,has_buildings_prefix,id_town,name_town,short_name_town,search_index_town,is_updated_town,is_actual_town,has_buildings_town,id_geonim,type_id_geonim,name_geonim,short_name_geonim,is_updated_geonim,is_actual_geonim,only_name_geonim,id_geonimtype,name_geonimtype,short_name_geonimtype,is_updated_geonimtype,is_actual_geonimtype,id_area,type_id_area,name_area,short_name_area,is_updated_area,is_actual_area,only_name_area,id_areatype,name_areatype,short_name_areatype,is_updated_areatype,is_actual_areatype,name_district_full,all_in_field,город,улица,дом,поселок,проспект,корпус,Литера,муниципальный район,набережная,шоссе,парк,переулок,площадь,аллея,,линия,автодорога,микрорайон,деревня,проезд,квартал,бульвар,станция,район,территория,муниципальный округ,мост,тупик,область,село,поселок городского типа,округ,волость,ner_city,ner_street,ner_house,ner_corpus,ner_liter,ner_district,ner_area,ner_subject
61150,57875,15941,34,61,NaN,NaN,NaN,NaN,"г.Санкт-Петербург, Шуваловский проспект, дом 61",True,True,Жилое,69.0,"г.Санкт-Петербург, Шуваловский пр., д. 61",NaN,NaN,34,Приморский,True,True,15941,36.0,763.0,NaN,NaN,"г.Санкт-Петербург, Шуваловский проспект","г.Санкт-Петербург, Шуваловский пр.","'г':1,7 'петербург':4,10 'проспект':6,12 'санкт':3,9 'санкт-петербург':2,8 'шуваловск':11 'шуваловский':5",True,True,15.0,True,36.0,г.Санкт-Петербург,г.Санкт-Петербург,"'г':1,5 'петербург':4,8 'санкт':3,7 'санкт-петербург':2,6",True,True,True,763.0,10.0,Шуваловский проспект,Шуваловский пр.,True,True,Шуваловский,10.0,проспект,пр.,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Приморский район,"г.Санкт-Петербург, Шуваловский проспект, дом 61, Жилое, Приморский район",САНКТ-ПЕТЕРБУРГ,NaN,61,NaN,ШУВАЛОВСКИЙ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,г.Санкт-Петербург,Шуваловский,61,NaN,NaN,Приморский,NaN,NaN
78451,76055,7907,34,77,NaN,NaN,NaN,NaN,"г.Санкт-Петербург, Школьная улица, дом 77",True,True,Неизвестно,65.0,"г.Санкт-Петербург, Школьная ул., д. 77",197187.0,NaN,34,Приморский,True,True,7907,36.0,1319.0,NaN,NaN,"г.Санкт-Петербург, Школьная улица","г.Санкт-Петербург, Школьная ул.","'г':1,7 'петербург':4,10 'санкт':3,9 'санкт-петербург':2,8 'улиц':12 'улица':6 'школьн':11 'школьная':5",True,True,15.0,True,36.0,г.Санкт-Петербург,г.Санкт-Петербург,"'г':1,5 'петербург':4,8 'санкт':3,7 'санкт-петербург':2,6",True,True,True,1319.0,12.0,Школьная улица,Школьная ул.,True,True,Школьная,12.0,улица,ул.,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Приморский район,"г.Санкт-Петербург, Школьная улица, дом 77, Неизвестно, Приморский район",САНКТ-ПЕТЕРБУРГ,ШКОЛЬНАЯ,77,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,г.Санкт-Петербург,Школьная,77,NaN,NaN,Приморский,NaN,NaN
91078,89171,7907,34,76,NaN,NaN,NaN,NaN,"г.Санкт-Петербург, Школьная улица, дом 76",True,True,Неизвестно,66.0,"г.Санкт-Петербург, Школьная ул., д. 76",197183.0,NaN,34,Приморский,True,True,7907,36.0,1319.0,NaN,NaN,"г.Санкт-Петербург, Школьная улица","г.Санкт-Петербург, Школьная ул.","'г':1,7 'петербург':4,10 'санкт':3,9 'санкт-петербург':2,8 'улиц':12 'улица':6 'школьн':11 'школьная':5",True,True,15.0,True,36.0,г.Санкт-Петербург,г.Санкт-Петербург,"'г':1,5 'петербург':4,8 'санкт':3,7 'санкт-петербург':2,6",True,True,True,1319.0,12.0,Школьная улица,Школьная ул.,True,True,Школьная,12.0,улица,ул.,True,True,NaN,NaN,NaN,NaN,NaN

In [170]:
cb_test = (
    pd.read_csv("../data/raw/datasets/dataset_1.csv")
    .drop(columns=['Unnamed: 0'])
    .dropna()
    .reset_index(drop=True)
    .astype({'target_building_id': 'int'})
)
cb_test

,address,target_building_id,target_address
0,"196641, Санкт-Петербург г, Школьная ул, д.5",76337,"Санкт-Петербург, город Павловск, Пязелево, Школьная улица, дом 5, литера А"
1,"Санкт-Петербург (г.), Железнодорожный (пр-кт.), д.14 2 1",220011,"г.Санкт-Петербург, Железнодорожный проспект, дом 14, корпус 2, строение 1"
2,"198325, Санкт-Петербург г, Красное Село г, Театральная (Дудергоф) ул, д.9",109979,"Санкт-Петербург, город Красное Село, Дудергоф, Театральная улица, дом 9, литера А"
3,"195256, Санкт-Петербург (г.), Верности (ул.), д.28, Строение 2",72112,"г.Санкт-Петербург, улица Верности, дом 28, корпус 2, литера А"
4,"194291, Санкт-Петербург (г.), Кустодиева (ул.), д.1, Строение А",186531,"г.Санкт-Петербург, улица Кустодиева, дом 1, литера А"
...,...,...,...
1133,"195197, Санкт-Петербург (г.), Федосеенко (ул.), д.37, Строение А",30906,"г.Санкт-Петербург, улица Федосеенко, дом 37, литера А"
1134,"193232, Санкт-Петербург г, Большевиков пр-кт, д.37, Строение 2",51342,"г.Санкт-Петербург, проспект Большевиков, дом 37, корпус 2, литера З"
1135,"192281, Санкт-Петербург г, Купчинская ул, д.14, Строение 2",33897,"г.Санкт-Петербург, Купчинская улица, дом 14, корпус 2, литера А"
1136,"196607, Санкт-Петербург г, Пушкин г, Ленинградская ул, д.17",40952,"Санкт-Петербург, город Пушкин, Ленинградская улица, дом 17, литера А"


In [267]:
def multiple_advanced_matching_scores(df, embeddings: np.array, sentences):
    res = pd.DataFrame()
    for index, row in df.iterrows():
        # print(row)
        i_address = row.address.replace('(', '').replace(')', '')
        i_df = get_advanced_matching_scores(i_address, embeddings, sentences)
        i_df.loc[:, 'y_correct'] = i_df.id_building == row.target_building_id
        res = pd.concat([res, i_df])
    return res.reset_index(drop=True)

multiple_advanced_matching_scores(cb_test.iloc[:10], np.array(embeddings), for_embeddings)

,id_building,address,scores,lev_distance,score_ner_city,score_ner_street,score_ner_house,original_query,y_correct
0,89171,"г.Санкт-Петербург, Школьная улица, дом 76",0.816349,18.0,0.918204,0.764108,0.478460,"196641, Санкт-Петербург г, Школьная ул, д.5",False
1,57875,"г.Санкт-Петербург, Шуваловский проспект, дом 61",0.820311,32.0,0.918204,1.000000,0.483058,"196641, Санкт-Петербург г, Школьная ул, д.5",False
2,76055,"г.Санкт-Петербург, Школьная улица, дом 77",0.831581,18.0,0.918204,1.000000,0.532030,"196641, Санкт-Петербург г, Школьная ул, д.5",False
3,107383,"г.Санкт-Петербург, Кузнечный переулок, дом 14б, литера Ж",0.903604,33.0,0.918204,0.509945,0.446655,"Санкт-Петербург г., Железнодорожный пр-кт., д.14 2 1",False
4,220198,"г.Санкт-Петербург, Железнодорожный проспект, дом 14, корпус 1, строение 1",0.911595,32.0,0.918204,0.487437,0.582745,"Санкт-Петербург г., Железнодорожный пр-кт., д.14 2 1",False
5,220011,"г.Санкт-Петербург, Железнодорожный проспект, дом 14, корпус 2, строение 1",0.917191,31.0,0.918204,0.487437,0.582745,"Санкт-Петербург г., Железнодорожный пр-кт., д.14 2 1",True
6,210842,"г.Санкт-Петербург, Афанасьевская улица, дом 21, корпус 5, строение 8",0.868495,53.0,0.918204,0.584699,0.294820,"198325, Санкт-Петербург г, Красное Село г, Театральная Дудергоф ул, д.9",False
7,135512,"г.Санкт-Петербург, Софийская улица, дом 79, корпус 2, литера Г5",0.869863,50.0,0.918204,0.584699,0.502145,"198325, Санкт-Петербург г, Красное Село г, Театральная Дудергоф ул, д.9",False
8,135527,"г.Санкт-Петербург, Софийская улица, дом 81, корпус 4, литера Г5",0.871385,50.0,0.918204,0.567020,0.514397,"198325, Санкт-Петербург г, Красное Село г, Театральная Дудергоф ул, д.9",False
9,214220,"г.Санкт-Петербург, Богатырский проспект, дом 54/32, строение 2",0.867504,36.0,0.918204,0.634248,0.339381,"195256, Санкт-Петербург г., Верности ул., д.28, Строение 2",False


In [ ]:
QUERY = "г.СПб,  корс 2, Грзовой пр."
get_address_ner_objects(QUERY)

In [212]:
# QUERY = "196641, Санкт-Петербург г, Школьная ул, д.5"
# get_address_ner_objects(QUERY)

{'город': 'САНКТ-ПЕТЕРБУРГ', 'улица': 'ШКОЛЬНАЯ', 'дом': '5'}

In [174]:
# get_advanced_matching_scores(cb_test.iloc[0, 0], np.array(embeddings), for_embeddings)

In [322]:
from catboost import CatBoostClassifier, Pool, metrics
from sklearn.model_selection import train_test_split
# y- True / False (correct id or not)
# x - scores <-> score_net_liter

exp_ds = multiple_advanced_matching_scores(cb_test.iloc[:100], np.array(embeddings), for_embeddings).drop(columns=['address', 'id_building'])
exp_ds['y_correct'] = exp_ds['y_correct'].astype(int)
X = exp_ds.drop(columns=['y_correct'])
Y = exp_ds['y_correct']

display(exp_ds.groupby('y_correct').scores.count())

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42, shuffle=True)
X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test, test_size=0.33, random_state=42, shuffle=True)

y_correct
0    279
1    5  
Name: scores, dtype: int64

In [300]:
X_train.dtypes

scores              float32
lev_distance        float64
score_ner_city      float32
score_ner_street    float32
score_ner_house     float32
original_query      object 
dtype: object

In [319]:
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, classification_report


def catboost_model_classifier(x_train, x_test, y_train, y_test):
    # categorical_features_indices = np.where(x_train.dtypes != np.float)[0]
    categorical_features_indices = np.where(x_train.dtypes == 'object')[
        0]  # X_train.select_dtypes('category').columns.to_list()

    cb_model = CatBoostClassifier(
        # custom_loss=metrics.Accuracy(),
        loss_function='Logloss',
        random_seed=42,
        logging_level='Silent',
        # custom_metric=['CrossEntropy', 'MultiClass'],
        max_depth=8,
        iterations=200,
        # scale_pos_weight=26
        auto_class_weights='Balanced',
        # eval_metric=[metrics.Precision(), metrics.Recall(), metrics.F1(), metrics.TotalF1(), metrics.Accuracy()]
    )

    cb_model.fit(
        x_train, y_train,
        eval_set=(x_test, y_test),
        cat_features=categorical_features_indices,
        plot=True
    )

    return cb_model

In [320]:
model = catboost_model_classifier(X_train, X_test, y_train, y_test)
model.save_model('../models/CatBoostClassifier')
y_pred_proba = model.predict_proba(X_valid)
y_pred = model.predict(X_valid)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [321]:
print(classification_report(y_pred, y_valid))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         4

    accuracy                           1.00         4
   macro avg       1.00      1.00      1.00         4
weighted avg       1.00      1.00      1.00         4


In [ ]:
def get_advanced_pred(query, embeddings, for_embeddings):
    model = CatBoostClassifier()
    model.load_model('../models/CatBoostClassifier')
    data = get_advanced_matching_scores(
        query, np.array(embeddings), for_embeddings
    ).drop(columns=["address", "id_building"])
    result = model.predict(data)
    return result
get_advanced_pred(QUERY, embeddings, for_embeddings)

# Step 6. Prepare data for API

In [24]:
def get_best_matches_json(query: str, embeddings, for_embeddings):
    prediction = get_best_matches(query, 1, embeddings, for_embeddings).rename(columns={'address': 'target_address', 'id_building': 'target_building_id'})
    pred_json = prediction.to_json()
    return prediction

get_best_matches_json("Санкт-Петербург г., Железнодорожный пр-кт., д.14 2 1", np.array(embeddings), for_embeddings)

,id_building,address,scores,lev_distance
161476,220011,"г.Санкт-Петербург, Железнодорожный проспект, дом 14, корпус 2, строение 1",0.917191,31.0


In [ ]:
#г.Санкт-Петербург, Железнодорожный проспект, дом 14, корпус 2
#г.Санкт-Петербург, Железнодорожный проспект, корпус 2
#г.Санкт-Петербург, Железнодорожный пр., корпус 2
#г.Санкт-Петербург, Железнодожный пр., корпус 2
# г.СПб, Железнодожный пр., корпус 2
г.СПБ, Железнодорожный проспект, дом 14 - 2
#

# г.Санкт-Петербург, Грузовой проезд, дом 17, корпус 2
# г.СПб,  корпус 2, Грузовой проезд
# г.СПб,  корпус 2, Грузовой пр.
г.СПб,  корпус 2, Грзовой пр.